# Fundamental tensor decompositions.
### Last modification (05.06.2018)

In this tutorial we provide a theoretical backgound on the fundamental tensor decompositions of multidimensional arrays and show how these data algorithms can be used with [hottbox](https://github.com/hottbox/hottbox) through **CPD**, **HOSVD**, **HOOI** and **TTSVD** classes.

More details on **CPD**, **HOSVD**, **HOOI** and **TTSVD** classes can be found on the [documentation page](https://hottbox.github.io/stable/api/hottbox.algorithms.decomposition).

**Note:** this tutorial assumes that you are familiar with the basics of tensor algebra, tensor representaitons in different forms and the corresponding conventional notation. If you are new to these topics, check out our previous tutorials: [tutorial_1](https://github.com/hottbox/hottbox-tutorials/blob/master/1_N-dimensional_arrays_and_Tensor_class.ipynb) and [tutorial_2](https://github.com/hottbox/hottbox-tutorials/blob/master/2_Efficient_representations_of_tensors.ipynb).

**Requirements:** ``hottbox==0.1.3``

**Authors:** 
Ilya Kisil (ilyakisil@gmail.com); 
Giuseppe G. Calvi (ggc115@ic.ac.uk)

In [1]:
import numpy as np
from hottbox.core import Tensor, residual_tensor
from hottbox.algorithms.decomposition import TTSVD, HOSVD, HOOI, CPD
from hottbox.metrics import residual_rel_error

# Tensor decompositions and their API

In [previous tutorial](https://github.com/hottbox/hottbox-tutorials/blob/master/2_Efficient_representations_of_tensors.ipynb), we have introduced various efficient representations of the multi-dimensional arrays (tensors) and how they can be created using the **`hottbox`** API. Here were show how these representations can obtained for a given tensor.

For these purposes, the following algorithms have been implemented in **``hottbox>=0.1.2``**:

- CPD: produces instance of **TensorCPD** class
- HOSVD: produces instance of **TensorTKD** class
- HOOI: produces instance of **TensorTKD** class
- TTSVD: produces instance of **TensorTT** class

By analogy with the computation algorithms in **`sklearn`**, you first need to create an instance of this algorithm. Then you use its method **`decompose`** in order to obtain an efficient representation of the original tensor. See [tutorial_2](https://github.com/hottbox/hottbox-tutorials/blob/master/2_Efficient_representations_of_tensors.ipynb) for more information on various efficient resentations of multi-dimensional arrays. For simplicity and ease of visualisation, the following matrial is provided for the tensors of order $3$, but can be easily generalised to a case of $N$-th order.

In all computational examples below we will decompose the same 3-D array with randomly generated values, while all algorithms will be initialised with default parameters.

In [2]:
np.random.seed(0)
I, J, K = 5, 6, 7

array_3d = np.random.rand(I * J * K).reshape((I, J, K)).astype(np.float)

tensor = Tensor(array_3d)
print(tensor)

This tensor is of order 3 and consists of 210 elements.
Sizes and names of its modes are (5, 6, 7) and ['mode-0', 'mode-1', 'mode-2'] respectively.


# Canonical Polyadic Decomposition (CPD)
![tensorcpd](./images/TensorCPD.png)
## Theoretical background

The **Canonical Polyadic Decomposition (CPD)** (also referred to as PARAFAC or CANDECOMP) is an algorithms that factorizes an $3$-rd order tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I \times J \times K}$ into a linear combination of terms $\mathbf{\underline{X}}_r = \mathbf{a}_r \circ \mathbf{b}_r \circ \mathbf{c}_r$, which are rank-$1$ tensors. In other words the tensor $\mathbf{\underline{X}}$ is decomposed as

$$
\begin{equation}
\begin{aligned}
\mathbf{\underline{X}} & \simeq \sum_{r=1}^{R} \lambda_r \mathbf{a}_r \circ \mathbf{b}_r \circ \mathbf{c}_r\\
& = \mathbf{\underline{\Lambda}} \times_1 \mathbf{A} \times_2 \mathbf{B} \times_3 \mathbf{C}\\
& = \Big[    \mathbf{\underline{\Lambda}} ;  \mathbf{A},  \mathbf{B}, \mathbf{C}      \Big]
\end{aligned}
\end{equation}
$$

where 

- $\mathbf{\underline{\Lambda}}$ is an $3$-rd order core tensor having $\lambda_r$ as entries in positions $\mathbf{\underline{\Lambda}}[i, j, k]$, where $i = j = k$, and zeroes elsewhere

- $\mathbf{A}, \mathbf{B}, \mathbf{C}$ are factor matrix obtained as the concatenation of the corresponding factor vectors, i.e  $ \mathbf{A} = \Big[    \mathbf{a}_1 \mathbf{a}_2  \cdots \mathbf{a}_R   \Big] $ 

Assuming the kruskal rank is fixed, there are many algorithms to compute a CPD. The most popular aproach is via the alternating least squares (ALS) method. The goal is to find such CP represenation $[ \mathbf{\underline{\Lambda}} ; \mathbf{A}, \mathbf{B}, \mathbf{C} ]$ which provides the best approximation of the original tensor $\mathbf{\underline{X}}$:

$$
\text{min} \| \mathbf{\underline{X}} - [ \mathbf{\underline{\Lambda}} ; \mathbf{A}, \mathbf{B}, \mathbf{C} ] \|^2_F
$$

The alternating least squares approach fixes $\mathbf{B}$ and $\mathbf{C}$ to solve for $\mathbf{A}$, then fixes $\mathbf{A}$ and $\mathbf{C}$ to solve for $\mathbf{B}$, then fixes $\mathbf{A}$ and $\mathbf{B}$ to solve for $\mathbf{C}$, and continues to repeat the
entire procedure until some convergence criterion is satisfied.


## CPD class in hottbox

In **`hottbox`**, the CPD-ALS algorithm is implemented by the **`CPD`** class. Despite of the parameters used to initialise this algorithm, it outputs an instance of **`TensorCPD`** class after each call of the **`decompose`** method. This method takes an object of **`Tensor`** class and desired value of kruskal rank passed as a tuple of length 1. 

**Note:** the Kruskal rank is passed as a tuple so to keep the same format with other algorithms for tensor decompositions.

In [3]:
alg = CPD()
alg

CPD(epsilon=0.01, init='svd', max_iter=50, random_state=None, tol=0.0001,
    verbose=False)

In [4]:
kruskal_rank = (5,)

tensor_cpd = alg.decompose(tensor, rank=kruskal_rank)
print("\tOutput of the {} algorithm:".format(alg.name))
print(tensor_cpd)

print('\n\tFactor matrices')
for mode, fmat in enumerate(tensor_cpd.fmat):
    print('Mode-{} factor matrix is of shape {}'.format(mode, fmat.shape))
    
print('\n\tCore tensor')
print(tensor_cpd.core)

	Output of the CPD algorithm:
Kruskal representation of a tensor with rank=(5,).
Factor matrices represent properties: ['mode-0', 'mode-1', 'mode-2']
With corresponding latent components described by (5, 6, 7) features respectively.

	Factor matrices
Mode-0 factor matrix is of shape (5, 5)
Mode-1 factor matrix is of shape (6, 5)
Mode-2 factor matrix is of shape (7, 5)

	Core tensor
This tensor is of order 3 and consists of 125 elements.
Sizes and names of its modes are (5, 5, 5) and ['mode-0', 'mode-1', 'mode-2'] respectively.


As we can see, the produced object of the **`TensorCPD`** class also contains general information about the underlying tensor, such as its shape, order etc, which can be accessed through the corresponding properties

In [5]:
full_shape = tensor_cpd.ft_shape
order = tensor_cpd.order
print('The shape of the underlying tensor is {}'.format(full_shape))
print('The order of the underlying tensor is {}'.format(order))

The shape of the underlying tensor is (5, 6, 7)
The order of the underlying tensor is 3


# Tucker Decomposition

![tensortkd](./images/TensorTKD.png)

**Tucker Decomposition** represents a given tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I \times J \times K}$ if the form of a dense core tensor $\mathbf{\underline{G}}$ with multi-linear rank $(Q, R, P)$ and a set of
factor matrices $\mathbf{A} \in \mathbb{R}^{I \times Q}, \mathbf{B} \in \mathbb{R}^{J \times R}$ and $\mathbf{C} \in
\mathbb{R}^{K \times P}$ as illustrated above. In other words, the tensor $\mathbf{\underline{X}}$ can represented in tucker form as

$$
\begin{equation}
\begin{aligned}
\mathbf{\underline{X}} & \simeq \sum_{q=1}^{Q} \sum_{r=1}^{R} \sum_{p=1}^{P} g_{qrp} \mathbf{a}_q \circ \mathbf{b}_r \circ \mathbf{c}_p\\
& = \mathbf{\underline{G}} \times_1 \mathbf{A} \times_2 \mathbf{B} \times_3 \mathbf{C}\\
& = \Big[    \mathbf{\underline{G}} ;  \mathbf{A},  \mathbf{B}, \mathbf{C}      \Big]
\end{aligned}
\end{equation}
$$

On practice, there exist several algorithms to represent a given tensor in the Tucker format. The two most used ones are Higher Order Singular Value Decomposition (HOSVD), and Higher Order Orthogonal Iteration (HOOI), which are implemented through the **`HOSVD`** and **`HOOI`** classes respectively.

## Higher Order Singular Value Decomposition (HOSVD)

Consider an $3$-rd order tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I \times J \times K}$, decomposed in the Tucker format as

$$
\mathbf{\underline{X}} = \mathbf{\underline{G}} \times_1 \mathbf{A} \times_2 \mathbf{B} \times_3 \mathbf{C}
$$

The HOSVD is a special case of the Tucker decomposition, in which all the factor matrices are constrained to be orthogonal. They are computed as truncated version of the left singular matrices of all possible mode-$n$ unfoldings of tensor $\mathbf{\underline{X}}$:

$$
\begin{aligned}
\mathbf{X}_{(1)} &= \mathbf{U}_1  \mathbf{\Sigma}_1 \mathbf{V}_1^T \quad \rightarrow \quad \mathbf{A} = \mathbf{U}_1[1:R_1]\\
\mathbf{X}_{(2)} &= \mathbf{U}_2  \mathbf{\Sigma}_2 \mathbf{V}_2^T \quad \rightarrow \quad \mathbf{B} = \mathbf{U}_2[1:R_2] \\
\mathbf{X}_{(3)} &= \mathbf{U}_3  \mathbf{\Sigma}_3 \mathbf{V}_3^T \quad \rightarrow \quad \mathbf{C} = \mathbf{U}_3[1:R_3] \\
\end{aligned}
$$

For a general order-$N$ tensor, the $N$-tuple $(R_1, \ldots, R_N)$ is called the **multi-linear rank** and provides flexibility in compression and approximation of the original tensor. For our order-$3$ tensor in the multilinear rank is therefore $(R_1, R_2, R_3)$. After factor matrices are obtained, the core tensor $\mathbf{\underline{G}}$ is computed as
$$
\mathbf{\underline{G}} = \mathbf{\underline{X}} \times_1 \mathbf{A}^T \times_2 \mathbf{B}^T \times_3 \mathbf{C}^T        
$$

## HOSVD class in hottbox

In **`hottbox`**, the HOSVD algorithm is implemented by the **`HOSVD`** class. Despite of the parameters used to initialise this algorithm, it outputs an instance of **`TensorTKD`** class after each call of the **`decompose`** method. This method takes an object of **`Tensor`** class and desired values of multi-linear rank passed as a tuple. 

In [6]:
alg = HOSVD()
alg

HOSVD(process=(), verbose=False)

In [7]:
ml_rank = (4, 5, 6)
tensor_tkd_hosvd = alg.decompose(tensor, ml_rank)
print("\tOutput of the {} algorithm:".format(alg.name))
print(tensor_tkd_hosvd)

print('\n\tFactor matrices')
for mode, fmat in enumerate(tensor_tkd_hosvd.fmat):
    print('Mode-{} factor matrix is of shape {}'.format(mode, fmat.shape))
    
print('\n\tCore tensor')
print(tensor_tkd_hosvd.core)

	Output of the HOSVD algorithm:
Tucker representation of a tensor with multi-linear rank=(4, 5, 6).
Factor matrices represent properties: ['mode-0', 'mode-1', 'mode-2']
With corresponding latent components described by (5, 6, 7) features respectively.

	Factor matrices
Mode-0 factor matrix is of shape (5, 4)
Mode-1 factor matrix is of shape (6, 5)
Mode-2 factor matrix is of shape (7, 6)

	Core tensor
This tensor is of order 3 and consists of 120 elements.
Sizes and names of its modes are (4, 5, 6) and ['mode-0', 'mode-1', 'mode-2'] respectively.


As we can see, the produced object of the **`TensorTKD`** class also contains general information about the underlying tensor, such as its shape, order etc, which can be accessed through the corresponding properties

In [8]:
full_shape = tensor_tkd_hosvd.ft_shape
order = tensor_tkd_hosvd.order
print('The shape of the underlying tensor is {}'.format(full_shape))
print('The order of the underlying tensor is {}'.format(order))

The shape of the underlying tensor is (5, 6, 7)
The order of the underlying tensor is 3


## Higher Order Orthogonal Iteration (HOOI)

HOOI algorithm is another special case of the Tuker decomposition. Like HOSVD, it decomposes a tensor into a dense core tensor and orthogonal factor matrices. The difference between the two lies in the fact that in HOOI the factor matrices are optimized iteratively using an Alternating Least Squares (ALS) approach. (In practice HOSVD is usually used within HOOI to initialize the factor matrices). In other words, the tucker representation $[ \mathbf{\underline{G}};\mathbf{A}^{(1)}, \mathbf{A}^{(2)}, \cdots,\mathbf{A}^{(N)} ]$ of the given tensor $\mathbf{\underline{X}}$ is obtained through the HOOI as follows

$$
\begin{aligned}
&\mathbf{\underline{Y}} = \mathbf{\underline{X}} \times_1 \mathbf{A}^{(1)T} \times_2 \cdots \times_{n-1} \mathbf{A}^{(n-1)T} \times_{n+1} \mathbf{A}^{(n+1)} \times \cdots \times_N \mathbf{A}^{(N)} \\
&\mathbf{A}^{(n)} \leftarrow R_n \text{ leftmost singular vectors of } \mathbf{Y}_{(n)}
\end{aligned}
$$

The above is repeated until convergence, then the core tensor $\mathbf{\underline{G}} \in \mathbb{R}^{R_1 \times R_2 \times \cdots \times R_N}$ is computed as

$$
\mathbf{\underline{G}} = \mathbf{\underline{X}} \times_1 \mathbf{A}^{(1)T}  \times_2 \mathbf{A}^{(2)T} \times_3 \cdots  \times_N \mathbf{A}^{(N)T}
$$

## HOOI class in hottbox

In **`hottbox`**, the HOOI algorithm is implemented by the **`HOOI`** class. Despite of the parameters used to initialise this algorithm, it outputs an instance of **`TensorTKD`** class after each call of the **`decompose`** method. This method takes an object of **`Tensor`** class and desired values of multi-linear rank passed as a tuple. 

In [9]:
alg = HOOI()
alg

HOOI(epsilon=0.01, init='hosvd', max_iter=50, process=(),
     random_state=None, tol=0.0001, verbose=False)

In [10]:
ml_rank = (4, 5, 6)
tensor_tkd_hooi = alg.decompose(tensor, ml_rank)
print("\tOutput of the {} algorithm:".format(alg.name))
print(tensor_tkd_hooi)

print('\n\tFactor matrices')
for mode, fmat in enumerate(tensor_tkd_hooi.fmat):
    print('Mode-{} factor matrix is of shape {}'.format(mode, fmat.shape))
    
print('\n\tCore tensor')
print(tensor_tkd_hooi.core)

	Output of the HOOI algorithm:
Tucker representation of a tensor with multi-linear rank=(4, 5, 6).
Factor matrices represent properties: ['mode-0', 'mode-1', 'mode-2']
With corresponding latent components described by (5, 6, 7) features respectively.

	Factor matrices
Mode-0 factor matrix is of shape (5, 4)
Mode-1 factor matrix is of shape (6, 5)
Mode-2 factor matrix is of shape (7, 6)

	Core tensor
This tensor is of order 3 and consists of 120 elements.
Sizes and names of its modes are (4, 5, 6) and ['mode-0', 'mode-1', 'mode-2'] respectively.


As we can see, the produced object of the **`TensorTKD`** class also contains general information about the underlying tensor, such as its shape, order etc, which can be accessed through the corresponding properties

In [11]:
full_shape = tensor_tkd_hooi.ft_shape
order = tensor_tkd_hooi.order
print('The shape of the underlying tensor is {}'.format(full_shape))
print('The order of the underlying tensor is {}'.format(order))

The shape of the underlying tensor is (5, 6, 7)
The order of the underlying tensor is 3


# Tensor Train Decomposition via SVD

![tensortt](./images/TensorTT.png)

## Theoretical background

**Tensor train decomposition** represents a given tensor a set of sparsely interconnected lower-order tensors and factor matrices. Mathematically speaking, the obtained TT representation of an $N$-th order tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I_1 \times I_2 \times \cdots \times I_N}$ can be expressed as a TT as

$$
\begin{aligned}
\mathbf{\underline{X}}
&= \Big[  \mathbf{A}, \mathbf{\underline{G}}^{(1)}, \mathbf{\underline{G}}^{(2)}, \cdots, \mathbf{\underline{G}}^{(N-1)}, \mathbf{B}  \Big]\\
&= \mathbf{A} \times^1_2 \mathbf{\underline{G}}^{(1)}  \times^1_3 \mathbf{\underline{G}}^{(2)}   \times^1_3 \cdots \times^1_3 \mathbf{\underline{G}}^{(N-1)} \times^1_3 \mathbf{B} 
\end{aligned}
$$

Each element of a TT is generally referred to as **tt-core** with sizesof its dimensions: $\mathbf{A} \in \mathbb{R}^{I_1 \times R_1}$, $\mathbf{B} \in \mathbb{R}^{R_{N-1}\times I_N}$, $\mathbf{\underline{G}}^{(n)} \in \mathbb{R}^{R_n \times I_{n+1} \times R_{n+1}}$


The TTSVD algorithm involves iteratively performing a series of foldings and unfoldings on an original tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I_1 \times I_2 \times \cdots \times I_N}$ in conjunction with SVD. At every iteration a core $\mathbf{\underline{G}}^{(n)} \in \mathbb{R}^{R_n \times I_{n+1} \times R_{n+1}}$ is computed, where the TT-rank $(R_1, R_2, \dots, R_N)$ has been specified a priori. 

## TTSVD class in hottbox

In **`hottbox`**, the TTSVD algorithm is implemented by the **`TTSVD`** class. Despite of the parameters used to initialise this algorithm, it outputs an instance of **`TensorTT`** class after each call of the **`decompose`** method. This method takes an object of **`Tensor`** class and desired values of tt-rank passed as a tuple. 

In [12]:
alg = TTSVD()
alg

TTSVD(verbose=False)

In [13]:
tt_rank = (2,3)

tensor_tt = alg.decompose(tensor, tt_rank)
print("\tOutput of the {} algorithm:".format(alg.name))
print(tensor_tt)

for i, core in enumerate(tensor_tt.cores):
    print('\n\tTT-Core #{}'.format(i))
    print(core)

	Output of the TTSVD algorithm:
Tensor train representation of a tensor with tt-rank=(2, 3).
Shape of this representation in the full format is (5, 6, 7).
Physical modes of its cores represent properties: ['mode-0', 'mode-1', 'mode-2']

	TT-Core #0
This tensor is of order 2 and consists of 10 elements.
Sizes and names of its modes are (5, 2) and ['mode-0', 'mode-1'] respectively.

	TT-Core #1
This tensor is of order 3 and consists of 36 elements.
Sizes and names of its modes are (2, 6, 3) and ['mode-0', 'mode-1', 'mode-2'] respectively.

	TT-Core #2
This tensor is of order 2 and consists of 21 elements.
Sizes and names of its modes are (3, 7) and ['mode-0', 'mode-1'] respectively.


As we can see, the produced object of the **`TensorTT`** class also contains general information about the underlying tensor, such as its shape, order etc, which can be accessed through the corresponding properties

In [14]:
full_shape = tensor_tt.ft_shape
order = tensor_tt.order
print('The shape of the underlying tensor is {}'.format(full_shape))
print('The order of the underlying tensor is {}'.format(order))

The shape of the underlying tensor is (5, 6, 7)
The order of the underlying tensor is 3


# Evaluating results of tensor decompositions

For each result of the tensor decomposition we can compute a residual tensor and calculate relative error of approximation:
```python
    tensor_res = residual_tensor(tensor, tensor_cpd)
    rel_error = tensor_res.frob_norm / tensor.frob_norm        
```
Or can do it in one line:
```python
    rel_error = residual_rel_error(tensor, tensor_cpd)
```

In [15]:
tensor_cpd_res = residual_tensor(tensor, tensor_cpd)
print('\tResidual tensor')
print(tensor_cpd_res)

	Residual tensor
This tensor is of order 3 and consists of 210 elements.
Sizes and names of its modes are (5, 6, 7) and ['mode-0', 'mode-1', 'mode-2'] respectively.


In [16]:
rel_error = tensor_cpd_res.frob_norm / tensor.frob_norm 
print('Relative error of CPD approximation = {:.2f}'.format(rel_error))

rel_error = residual_rel_error(tensor, tensor_cpd)
print('Relative error of CPD approximation = {:.2f}'.format(rel_error))

Relative error of CPD approximation = 0.31
Relative error of CPD approximation = 0.31


In [17]:
rel_error = residual_rel_error(tensor, tensor_tkd_hosvd)
print('Relative error of HOSVD approximation = {:.2f}'.format(rel_error))

Relative error of HOSVD approximation = 0.21


In [18]:
rel_error = residual_rel_error(tensor, tensor_tkd_hooi)
print('Relative error of HOOI approximation = {:.2f}'.format(rel_error))

Relative error of HOOI approximation = 0.21


In [19]:
rel_error = residual_rel_error(tensor, tensor_tt)
print('Relative error of TT approximation = {:.2f}'.format(rel_error))

Relative error of TT approximation = 0.39


# Further reading list
- Tamara G. Kolda and Brett W. Bader, "Tensor decompositions and applications." SIAM REVIEW, 51(3):455–500, 2009.

- Lieven De Lathauwer, Bart De Moor, and Joos Vandewalle, "A multilinear singular value decomposition." SIAM journal on Matrix Analysis and Applications 21.4 (2000): 1253-1278.

- Ivan V. Oseledets,  "Tensor-train decomposition." SIAM Journal on Scientific Computing 33.5 (2011): 2295-2317.